En este codigo queremos hacer un analisis del entrenamiento para observa el efecto de mejora. Para eso vamos a procesar los trials, con la idea de hacer un ajuste tipo de curva de respuesta y luego ver como esa curva varia con el tiempo y el agregado de nuevos datos.

Lo primero que tenemos que hacer es recuperar los datos en formato amigable...

In [15]:
import json
import os
import pickle    
import pandas as pd

In [70]:
# Lo primero que hacemos en cargar los datos crudos

filename = 'DatosTransferencia/db.json'

# Transformamos el archivo en un json
with open(filename) as data_file:
    db = json.load(data_file)

# Buscamos los envios
db = db['Envio']
# Filtramos solo las convergencias
db = [elemento for elemento in db if elemento['tipo']=='RESULTADOS']

# Cargamos los datos del los usuarios 
with open('DatosTransferencia/datos.Alias', 'rb') as f:
    users = pickle.load(f)
    
# Seleccionamos solo los que nos interesan (Marco esta duplicado, hay que ver despues que onda...)
usersExp = [user for user in users if not user['ignore']]
usersIds = [user['id'] for user in usersExp]
usersMap = {}
for user in usersExp:
    usersMap[user['id']]=user['alias']

# Filtramos solo los datos de estos usuarios
db = [envio for envio in db if envio['objeto']['levelLog']['sesion']['user']['id'] in usersIds]

# Revisamos que no haya envios duplicados
instances = [envio['instance'] for envio in db]
len (instances) == len(set(instances))

# Pasamos al formato pandas
data = pd.DataFrame(db)

# Tiramos columnas innecesarias
data = data.drop(['estadoEnvio','id','instance','tipo'], axis=1)



#Creamos una funcion que nos extrae lo necesrio
def Extraer(data,columna):
    for key in data.iloc[0][columna]:
        data[key] = data[columna].map(lambda x: x[key])
    return data.drop([columna],axis=1)





# Expandimos el json del objeto
data = Extraer(data,'objeto')
data = data.drop(['class'],axis=1)


data = Extraer(data,'levelLog')
# Verificamos que origen y identificadorNivel son redundantes
sum(data['origen'] == data['identificadorNivel']) == len(data)
data = data.drop(['origen'], axis=1)

# Verificamos que haya un solo envio por nivel y renombramos la columna
len(data['instance'].unique()) == len(data)
data['levelInstance'] = data['instance']
data = data.drop(['instance'], axis=1)

# Extraemos la info del json de session
data = Extraer(data,'sesion')
# Tiramos data innecesaria
data = data.drop(['resourcesVersion','plataforma','codeVersion'], axis=1)

# Extraemos la info del user
data = Extraer(data,'user')
data['userId'] = data['id']
data = data.drop(['id'], axis=1)
data['Alias'] = data['userId'].map(usersMap)

In [72]:
userlist = data['Alias'].unique()

In [84]:
filtrado = data[data['Alias']==userlist[0]]
filtrado

,dinamica,identificadorNivel,levelInstance,sessionInstance,eleccion,faseDeExperimentoActiva,levelsJugados,userId,Alias
0,"{'aciertosAcumulados': 0, 'trialsPorNivel': 10...",ENTRENAMIENTOA30INICIAL,1473253986029,1473253943655,A30,Entrenamiento2,"[{'contexto': 'Tutorial', 'nivelAlFinalizar': ...",1473088663799,ExpT_Marco
1,"{'aciertosAcumulados': 0, 'trialsPorNivel': 50...",ENTRENAMIENTOA30MEDIO,1473254539616,1473253943655,A30,Entrenamiento2,"[{'contexto': 'Tutorial', 'nivelAlFinalizar': ...",1473088663799,ExpT_Marco
2,"{'aciertosAcumulados': 1, 'trialsPorNivel': 10...",ENTRENAMIENTOA30FINAL,1473254832057,1473253943655,A30,Entrenamiento2,"[{'contexto': 'Tutorial', 'nivelAlFinalizar': ...",1473088663799,ExpT_Marco
18,"{'aciertosAcumulados': 0, 'trialsPorNivel': 10...",ENTRENAMIENTOA30INICIAL,1473336820336,1473336796730,A30,Entrenamiento3,"[{'contexto': 'Tutorial', 'nivelAlFinalizar': ...",1473088663799,ExpT_Marco
19,"{'aciertosAcumulados': 0, 'trialsPorNivel': 50...",ENTRENAMIENTOA30MEDIO,1473337358151,1473336796730,A30,Entrenamiento3,"[{'contexto': 'Tutorial', 'nivelAlFinalizar': ...",1473088663799,ExpT_Marco
20,"{'aciertosAcumulados': 0, 'trialsPorNivel': 10...",ENTRENAMIENTOA30FINAL,1473337628696,1473336796730,A30,Entrenamiento3,"[{'contexto': 'Tutorial', 'nivelAlFinalizar': ...",1473088663799,ExpT_Marco
36,"{'aciertosAcumulados': 1, 'trialsPorNivel': 10...",ENTRENAMIENTOA30INICIAL,1473683434877,1473683211999,A30,Entrenamiento4,"[{'contexto': 'Tutorial', 'nivelAlFinalizar': ...",1473088663799,ExpT_Marco
37,"{'aciertosAcumulados': 1, 'trialsPorNivel': 50...",ENTRENAMIENTOA30MEDIO,1473683936794,1473683211999,A30,Entrenamiento4,"[{'contexto': 'Tutorial', 'nivelAlFinalizar': ...",1473088663799,ExpT_Marco
38,"{'aciertosAcumulados': 1, 'trialsPorNivel': 10...",ENTRENAMIENTOA30FINAL,1473684184399,1473683211999,A30,Entrenamiento4,"[{'contexto': 'Tutorial', 'nivelAlFinalizar': ...",1473088663799,ExpT_Marco
80,"{'aciertosAcumulados': 0, 'trialsPorNivel': 40...",TESTP30,1473958899244,1473958863535,A30,TestFinal,"[{'contexto': 'Tutorial', 'nivelAlFinalizar': ...",1473088663799,ExpT_Marco


In [76]:
filtrado

,dinamica,identificadorNivel,levelInstance,sessionInstance,eleccion,faseDeExperimentoActiva,levelsJugados,userId,Alias
0,"{'aciertosAcumulados': 0, 'trialsPorNivel': 10...",ENTRENAMIENTOA30INICIAL,1473253986029,1473253943655,A30,Entrenamiento2,"[{'contexto': 'Tutorial', 'nivelAlFinalizar': ...",1473088663799,ExpT_Marco
1,"{'aciertosAcumulados': 0, 'trialsPorNivel': 50...",ENTRENAMIENTOA30MEDIO,1473254539616,1473253943655,A30,Entrenamiento2,"[{'contexto': 'Tutorial', 'nivelAlFinalizar': ...",1473088663799,ExpT_Marco
2,"{'aciertosAcumulados': 1, 'trialsPorNivel': 10...",ENTRENAMIENTOA30FINAL,1473254832057,1473253943655,A30,Entrenamiento2,"[{'contexto': 'Tutorial', 'nivelAlFinalizar': ...",1473088663799,ExpT_Marco
18,"{'aciertosAcumulados': 0, 'trialsPorNivel': 10...",ENTRENAMIENTOA30INICIAL,1473336820336,1473336796730,A30,Entrenamiento3,"[{'contexto': 'Tutorial', 'nivelAlFinalizar': ...",1473088663799,ExpT_Marco
19,"{'aciertosAcumulados': 0, 'trialsPorNivel': 50...",ENTRENAMIENTOA30MEDIO,1473337358151,1473336796730,A30,Entrenamiento3,"[{'contexto': 'Tutorial', 'nivelAlFinalizar': ...",1473088663799,ExpT_Marco
20,"{'aciertosAcumulados': 0, 'trialsPorNivel': 10...",ENTRENAMIENTOA30FINAL,1473337628696,1473336796730,A30,Entrenamiento3,"[{'contexto': 'Tutorial', 'nivelAlFinalizar': ...",1473088663799,ExpT_Marco
36,"{'aciertosAcumulados': 1, 'trialsPorNivel': 10...",ENTRENAMIENTOA30INICIAL,1473683434877,1473683211999,A30,Entrenamiento4,"[{'contexto': 'Tutorial', 'nivelAlFinalizar': ...",1473088663799,ExpT_Marco
37,"{'aciertosAcumulados': 1, 'trialsPorNivel': 50...",ENTRENAMIENTOA30MEDIO,1473683936794,1473683211999,A30,Entrenamiento4,"[{'contexto': 'Tutorial', 'nivelAlFinalizar': ...",1473088663799,ExpT_Marco
38,"{'aciertosAcumulados': 1, 'trialsPorNivel': 10...",ENTRENAMIENTOA30FINAL,1473684184399,1473683211999,A30,Entrenamiento4,"[{'contexto': 'Tutorial', 'nivelAlFinalizar': ...",1473088663799,ExpT_Marco
80,"{'aciertosAcumulados': 0, 'trialsPorNivel': 40...",TESTP30,1473958899244,1473958863535,A30,TestFinal,"[{'contexto': 'Tutorial', 'nivelAlFinalizar': ...",1473088663799,ExpT_Marco


In [88]:
from scripts.db import *
from scripts.plots import *
from scripts.stats import *



In [90]:
users, db = dataProcesada()

AttributeError: 'module' object has no attribute 'PATHALIA'